# Avicenna: A Semantic Debugging Tool

**Avicenna** is a debugging tool designed to automatically determine the causes and conditions of program failures. This notebook provides an overview and demonstration of its capabilities.

**Avicenna** leverages both generative and predictive models to satisfy constraints over grammar elements and detect relations of input elements.
Our tool uses the [ISLa](https://github.com/rindPHI/isla) specification language to express complex failure circumstances as predicates over input elements.
**Avicenna** learns input properties that are common across failing inputs and employs a feedback loop to refine the current debugging diagnoses by systematic experimentation.
The result is crisp and precise diagnoses that closely match those determined by human experts, offering a significant advancement in the realm of automated debugging.


### Example Subject: The Calculator

To illustrate **Avicenna**’s capabilities, we start with a quick motivating example. First, let us introduce our program under test: The Calculator.

The Calculator is a Python program designed to evaluate various mathematical expressions, including arithmetic equations and trigonometric functions:

In [1]:
import math

def calculator(inp: str) -> float:
    """
        A simple calculator function that can evaluate arithmetic expressions
        and perform basic trigonometric functions and square root calculations.
    """
    return eval(
        str(inp), {"sqrt": math.sqrt, "sin": math.sin, "cos": math.cos, "tan": math.tan}
    )

We can test the calculator with some sample inputs:

In [2]:
# Evaluating the cosine of 2π
print(calculator('cos(6*3.141)'))

0.999993677717667


In [3]:
# Calculating the square root of 36
print(calculator('sqrt(6*6)'))

6.0


Each of these calls to the calculator will evaluate the provided string as a mathematical expression, and print the result.

### Defining an Oracle to classify failure-inducing Inputs

To identify bugs in The Calculator, we'll implement an oracle function. This function tests inputs and categorizes them as producing expected behavior (`OracleResult.PASSING`) or a bug (`OracleResult.FAILING`):

We import the `OracleResult` enumerated type from the `avicenna` library.

In [4]:
from avicenna.data import OracleResult

This allows us to define a function called **oracle**, which acts as an intermediary to handle and classify exceptions produced by the calculator function when given a certain input.

In [5]:
# Make sure you use the OracleResult from the evogfuzz library
from avicenna.data import OracleResult

def oracle(inp: str):
    """
    This function serves as an oracle or intermediary that catches and handles exceptions 
    generated by the 'calculator' function.
    It aims to determine whether an input triggers a bug in the 'calculator' function.

    Args:
        inp (str): The input string to be passed to the 'calculator' function.

    Returns:
        OracleResult: An enumerated type 'OracleResult' indicating the outcome of the function execution.
            - OracleResult.PASSING: Returned if the calculator function executes without any exception
            - OracleResult.FAILING: Returned if the calculator function raises a ValueError exception, indicating a potential bug.
    """
    try:
        calculator(inp)
    except ValueError as e:
        return OracleResult.FAILING
    return OracleResult.PASSING

This **oracle** function is used in the context of debugging to determine the behavior of various inputs on the program under test (in our case the _calculator_). When the calculator function behaves as expected (i.e., no exceptions occur), the **oracle** function returns `OracleResult.PASSING`. However, when the `calculator` function raises an unexpected exception, the **oracle** interprets this as a potential bug in the `calculator` and returns `OracleResult.FAILING`.

We can see this in action by testing a few initial inputs:


In [6]:
initial_inputs = ['sqrt(1)', 'cos(912)', 'tan(4)', 'sqrt(-3)']

for inp in initial_inputs:
    print(inp.ljust(30), oracle(inp))

sqrt(1)                        PASSING
cos(912)                       PASSING
tan(4)                         PASSING
sqrt(-3)                       FAILING


We see that `sqrt(-3)` results in the failure of our calculator program. 

In the following steps, we'll leverage Avicenna to pinpoint the root cause of this bug.
We'll employ Avicenna's capabilities to identify the root cause of the bug and provide potential fixes.
This will involve defining the input structure for our calculator and initializing Avicenna with the appropriate grammar, sample inputs, and oracle function.

### Defining the Input Format

First, we need to define the input format of the calculator with a grammar:

In [7]:
import string

grammar = {
    "<start>": ["<arith_expr>"],
    "<arith_expr>": ["<function>(<number>)"],
    "<function>": ["sqrt", "sin", "cos", "tan"],
    "<number>": ["<maybe_minus><onenine><maybe_digits><maybe_frac>"],
    "<maybe_minus>": ["", "-"],
    "<onenine>": [str(num) for num in range(1, 10)],
    "<digit>": list(string.digits),
    "<maybe_digits>": ["", "<digits>"],
    "<digits>": ["<digit>", "<digit><digits>"],
    "<maybe_frac>": ["", ".<digits>"],
}

The grammar provides a structured way to generate valid input strings for our calculator program. It defines patterns and rules that dictate how different elements can be combined to form syntactically correct mathematical expressions. Here's a breakdown of the key components of the grammar:

- `<start>`: The entry point for generating an expression. It signifies where the creation of an arithmetic expression begins.

- `<arith_expr>`: Represents a general arithmetic expression. For simplicity in this example, it's defined to consist of a function applied to a number, like `sin(3)` or `sqrt(9)`.

- `<function>`: Enumerates the mathematical functions our calculator can handle, including square root and trigonometric operations like sine, cosine, and tangent.

- `<number>`: Describes valid numbers for our calculator. This includes:
  - Negative values (denoted by `<maybe_minus>` which can be an empty string or a minus sign).
  - Whole numbers ranging from 1 to 9 (given by `<onenine>`).
  - Sequences of digits (represented by `<maybe_digits>` and `<digits>`).
  - Fractions or decimal numbers (expressed by `<maybe_frac>`).

This grammar acts as a blueprint, guiding the systematic generation of test cases for our calculator. By defining the rules and structures of valid inputs, it ensures that the generated expressions are meaningful and relevant for our debugging exercise.

### Using Avicenna

With the oracle, the grammar, and a failure-inducing input, we can use **Avicenna** to automatically infer properties over inputs, validate hypotheses, generate additional test cases, and finally producing precise and expressive diagnoses for the failure conditions.

In [8]:
from avicenna import Avicenna

avicenna = Avicenna(
    grammar,
    oracle,
    initial_inputs,
    max_iterations=4,
)

In [9]:
from typing import List, Tuple
from isla.language import Formula

from avicenna.diagnostic import Candidate

diagnoses: List[Candidate] = avicenna.explain()
# Avicenna returns a List of learned ISla Formula and the corresponding recall, precision, and specificity.

In the code above, we've created an instance of the Avicenna class and executed the debugging process by invoking the `explain()` method.
Avicenna will utilize its feedback loop to systematically probe and test the Calculator program, identify the root cause of the bug on the analysis of the bug's behavior.

This output is a symbolic representation - ISLa Constraints - of the root cause of the failure detected by Avicenna in the Calculator program. Here's a breakdown of what it means:


In [10]:
failure_diagnosis = diagnoses.pop(0)

In [11]:
from isla.language import ISLaUnparser

print(f"Avicenna determined the following constraints to describe the failure circumstances:\n")

print(ISLaUnparser(failure_diagnosis.formula).unparse())
print(f"Avicenna calculated a precision of {failure_diagnosis.precision()*100:.2f}% and \
a recall of {failure_diagnosis.recall()*100:.2f}%", end="\n\n")

Avicenna determined the following constraints to describe the failure circumstances:

(exists <function> elem in start:
   (= elem "sqrt") and
exists <maybe_minus> elem_0 in start:
  (= elem_0 "-"))
Avicenna calculated a precision of 100.00% and a recall of 100.00%



This output, expressed in first-order logic, is saying:

- For all numbers (elements of type `<number>` in the grammar), if the integer representation of the number is less than or equal to -1 (`<= (str.to.int elem) (str.to.int "-1")`), and
- There exists a function (an element of type `<function>` in the grammar) that equals to "sqrt" (`= elem_0 "sqrt"`),

then a bug is likely to occur.

In plain English, the output is indicating that the failure in our Calculator program occurs when trying to take the square root (`sqrt`) of a negative number (a number less than or equal to -1). 

This is consistent with our expectations, since the square root of a negative number is not defined in the realm of real numbers. Consequently, Python's `math.sqrt()` function, which we've used in our Calculator program, throws a `ValueError` when given a negative number as input.

With this information, we can address the issue in our Calculator program to prevent crashes when dealing with such inputs. We might decide to handle such errors gracefully or implement support for complex numbers, depending on the requirements of our program.

Remember, these results are generated based on the information provided to Avicenna, such as the grammar and the oracle function, as well as the results of Avicenna's systematic testing of the Calculator program. So the more accurate and comprehensive these inputs are, the more helpful Avicenna's outputs will be.


## Generating More Inputs from the diagnoses

Now that we obtained the ISLa formulas that describe the failure circumstances, we can use them to generate more inputs triggering that exact same behavior. To do so, we use the ISLaSolver:

The function `ISLaSolver.solve()` attempts to compute a solution to the given ISLa formula. It returns that solution, if any. This function can be called repeatedly to obtain more solutions until one of two exception types is raised: A `StopIteration` indicates that no more solution can be found; a `TimeoutError` is raised if a timeout occurred. After that, an exception will be raised every time.

<div class="alert alert-info">
[Info]: For more information about the <a href="https://github.com/rindPHI/isla">ISLa Sepcification language</a> and the <b>ISLaSolver</b>, have a look at the extensive <a href="https://isla.readthedocs.io/en/latest/index.html">Documentation</a>.
</div>

In [16]:
from isla.solver import ISLaSolver

generated_inputs = set()

for diagnosis in avicenna.get_best_candidates():
    solver = ISLaSolver(
        grammar,
        formula=diagnosis.formula,
        enable_optimized_z3_queries=True)
        
    for _ in range(20):
        try:
            inp = solver.solve()
            generated_inputs.add(inp)
            print(str(inp).ljust(30), oracle(inp))
        except Exception:
            continue

sqrt(-8.60)                    FAILING
sqrt(-57)                      FAILING
sqrt(-7132.9584)               FAILING
sqrt(-97054.6)                 FAILING
sqrt(-2.5)                     FAILING
sqrt(-4.6566078)               FAILING
sqrt(-300)                     FAILING
sqrt(-1.51795)                 FAILING
sqrt(-668.3)                   FAILING
sqrt(-5.3)                     FAILING
sqrt(-42)                      FAILING
sqrt(-6.890)                   FAILING
sqrt(-38.1)                    FAILING
sqrt(-888.7)                   FAILING
sqrt(-90.37)                   FAILING
sqrt(-3.561)                   FAILING
sqrt(-153)                     FAILING
sqrt(-2)                       FAILING
sqrt(-4)                       FAILING
sqrt(-90.203)                  FAILING
sqrt(-1)                       FAILING
sqrt(-1)                       FAILING
sqrt(-53.85942)                FAILING
sqrt(-20671.0)                 FAILING
sqrt(-78)                      FAILING
sqrt(-9)                 

In [19]:
from avicenna.data import Input

test_inputs_avicenna = [Input(tree=inp, oracle=oracle(inp)) for inp in generated_inputs]

In [20]:
for inp in test_inputs_avicenna:
    print(str(inp), inp.oracle)

sqrt(1) PASSING
sqrt(-1) FAILING
sqrt(-6.22) FAILING
sqrt(-36752) FAILING
sqrt(-2.3) FAILING
sqrt(-640) FAILING
sqrt(-5.11) FAILING
sqrt(-7.235) FAILING
sqrt(-4.699) FAILING
sqrt(-2.3) FAILING
sqrt(-2) FAILING
sqrt(-680.7) FAILING
sqrt(-1) FAILING
sqrt(-36.3) FAILING
sqrt(-5195.24) FAILING
sqrt(-9) FAILING
sqrt(-727) FAILING
sqrt(-2) FAILING
sqrt(-446.3) FAILING
sqrt(-885) FAILING
sqrt(-3.7) FAILING
sqrt(-90.37) FAILING
sqrt(-3.561) FAILING
sqrt(-153) FAILING
sqrt(-2) FAILING
sqrt(-4) FAILING
sqrt(-8.60) FAILING
sqrt(-57) FAILING
sqrt(-97054.6) FAILING
sqrt(-2.5) FAILING
sqrt(-7132.9584) FAILING
sqrt(-57) FAILING
sqrt(-1.92346) FAILING
sqrt(-850.81) FAILING
sqrt(-409.2) FAILING
sqrt(-29) FAILING
sqrt(-7) FAILING
sqrt(-3915.053) FAILING
sqrt(-67.0) FAILING
sqrt(-944.6) FAILING
sqrt(-8) FAILING
sqrt(-4.6566078) FAILING
sqrt(-300) FAILING
sqrt(-1.51795) FAILING
sqrt(-668.3) FAILING
sqrt(-5.3) FAILING
sqrt(-42) FAILING
sqrt(-6.890) FAILING
sqrt(-38.1) FAILING
sqrt(-888.7) FAILING
sqrt(-90.

## Summary:

In this notebook, we introduced **Avicenna**, a powerful semantic debugging tool designed to automatically determine the causes and conditions of program failures. Through the example of a simple calculator program, we showcased the following:

1. **Setting Up an Oracle**: We defined an intermediary function, termed as an 'oracle', which classifies the exceptions produced by our program into expected and unexpected categories.
2. **Grammar Definition**: A structured blueprint for generating valid inputs to the calculator was established.
3. **Automated Debugging with Avicenna**: Using the provided grammar, initial test cases, and the oracle, Avicenna systematically probed our calculator program and identified potential root causes for observed failures.
4. **Interpreting Results**: We decoded Avicenna's output, learning that the failure in our calculator program is triggered when computing the square root of a negative number.

# Are we able to Improve SFL?

In [32]:
from pathlib import Path

import tests4py.api as t4p
from tests4py import sfl

In [33]:
report = t4p.checkout(t4p.expression_1)

In [34]:
report

{
    "command": "checkout",
    "successful": true,
    "project": "expression_1",
    "location": "PosixPath('/Users/martineberlein/github/avicenna/notebooks/sfl/tmp/expression_1')"
}

In [35]:
src = report.location

In [36]:
dst = Path("tmp", "sfl")
sfl.sflkit_instrument(dst, events="line")


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


{
    "command": "sfl",
    "subcommand": "instrument",
    "successful": true,
    "project": "expression_1"
}

In [42]:
from isla.fuzzer import GrammarFuzzer
from debugging_benchmark.middle.middle import middle_grammar
from debugging_benchmark.expression.expression import expression_grammar

test_inputs = []
fuzzer = GrammarFuzzer(expression_grammar)

for inp in range(100):
    inp = fuzzer.fuzz_tree()
    test_inputs.append(inp)

In [45]:
from debugging_benchmark.expression.expression import ExpressionBenchmarkRepository

prog = ExpressionBenchmarkRepository().build()[0]
expression_oracle = prog.oracle

In [51]:
test_inputs_expression = [Input(inp, expression_oracle(inp)[0]) for inp in test_inputs]

In [53]:
for inp in test_inputs_expression:
    print(str(inp), inp.oracle)

failing_expression_inputs = [inp for inp in test_inputs_expression if inp.oracle == OracleResult.FAILING]
len(failing_expression_inputs)

((0)) PASSING
((~ 71 - ~ 26 - (0))) PASSING
(0 - ((0))) PASSING
(0 / 0 + 0 / ~ 3953) FAILING
1 - (0) PASSING
2 / 0 - 0 / ~ 3 FAILING
1997 PASSING
0 PASSING
((~ 88 * (((0)) * 62 * (((71))) + (7110)))) PASSING
((~ 50)) * (27) + ((0 - 0 + 0 / 0) - (0 / 2) - (0) - 0) + 0 FAILING
0 PASSING
(0) PASSING
(((((85469))))) PASSING
0 PASSING
((0)) PASSING
430 PASSING
~ 945 PASSING
0 * ~ 227314 PASSING
8 PASSING
(53463 * 0 / 33) PASSING
~ 3948544 PASSING
~ 6 PASSING
~ 689045 PASSING
((0) - ~ 5 + ((0 - 0 + (~ 8)) + 0 + (((0) - 0)) * (0) * 0 - ~ 3) + 0) - ~ 8 PASSING
(~ 58) PASSING
(((((0) + 0 / (0 - 0)))) - 0 - 0 - 0 + 0) FAILING
((0)) + ((~ 46)) PASSING
0 PASSING
(0) PASSING
(0) * 0 PASSING
(2 - (826)) + ((((0 - 0) / 0 - (0)))) FAILING
(((0 / 0) / 0)) - 0 + 0 + 3 FAILING
(((198218) - 0 - 71 + 1)) PASSING
(0 / ~ 8 * (0 * 0) + 0 + 0 * 0 + 0 / 0 * 0 * ~ 1) FAILING
0 * (0) PASSING
((~ 63)) PASSING
0 PASSING
(((0)) + (0) * (((0 / 0 / 0 - 0 * 0) + 0)) + 4) FAILING
1 - ~ 95 PASSING
0 PASSING
0 PASSING
0 P

20

In [83]:
test_inputs = failing_expression_inputs

In [84]:
import os
import tempfile

# test_inputs = ['1 / 10', '5 / 0']
# List to hold temporary file paths
temp_files = []

# Create temporary files for each string
for s in test_inputs:
    temp_file = tempfile.NamedTemporaryFile(delete=False)
    temp_file.write(str(s).encode())
    temp_file.close()  # Close so we can reopen it later
    temp_files.append(temp_file.name)

In [99]:
temp_files

['/var/folders/02/7jyqzsc129vc1fqtp6sqknnm0000gn/T/tmpk2nzcxjo',
 '/var/folders/02/7jyqzsc129vc1fqtp6sqknnm0000gn/T/tmpv3uji4sx',
 '/var/folders/02/7jyqzsc129vc1fqtp6sqknnm0000gn/T/tmp59t2w57_',
 '/var/folders/02/7jyqzsc129vc1fqtp6sqknnm0000gn/T/tmp5eybz1zd',
 '/var/folders/02/7jyqzsc129vc1fqtp6sqknnm0000gn/T/tmpiksuz0ov',
 '/var/folders/02/7jyqzsc129vc1fqtp6sqknnm0000gn/T/tmpqv3nwn9f',
 '/var/folders/02/7jyqzsc129vc1fqtp6sqknnm0000gn/T/tmppkg84znm',
 '/var/folders/02/7jyqzsc129vc1fqtp6sqknnm0000gn/T/tmpe0w65ano',
 '/var/folders/02/7jyqzsc129vc1fqtp6sqknnm0000gn/T/tmps_xxu6hg',
 '/var/folders/02/7jyqzsc129vc1fqtp6sqknnm0000gn/T/tmp5pky4eij',
 '/var/folders/02/7jyqzsc129vc1fqtp6sqknnm0000gn/T/tmpwsmrmqb7',
 '/var/folders/02/7jyqzsc129vc1fqtp6sqknnm0000gn/T/tmpphoy3rbq',
 '/var/folders/02/7jyqzsc129vc1fqtp6sqknnm0000gn/T/tmppmq6p6pr',
 '/var/folders/02/7jyqzsc129vc1fqtp6sqknnm0000gn/T/tmp3tvpt3gx',
 '/var/folders/02/7jyqzsc129vc1fqtp6sqknnm0000gn/T/tmp_bfl0wqk',
 '/var/folders/02/7jyqzsc

In [100]:
sfl.sflkit_systemtest(work_dir=dst, tests=temp_files)

{
    "command": "sfl",
    "subcommand": "events",
    "successful": true,
    "project": "expression_1"
}

In [101]:
report = sfl.sflkit_analyze(work_dir=dst, 
                            src=src,
                            predicates="line")

In [102]:
from sflkit.analysis.analysis_type import AnalysisType
from sflkit.analysis.spectra import Spectrum

suggestions = report.analyzer.get_sorted_suggestions(src,
            type_=AnalysisType.LINE,
            metric=Spectrum.Ochiai,)
suggestions

[[src/expression/expr/parse.py:57, src/expression/expr/arithmetic.py:32, src/expression/expr/arithmetic.py:33]:0.9534625892455924,
 [src/expression/expr/parse.py:53, src/expression/expr/parse.py:54]:0.8770580193070293,
 [src/expression/expr/parse.py:65, src/expression/expr/parse.py:66]:0.816496580927726,
 [src/expression/expr/parse.py:67]:0.7841515268245522,
 [src/expression/expr/arithmetic.py:8, src/expression/expr/arithmetic.py:9]:0.778498944161523,
 [src/expression/expr/arithmetic.py:14]:0.7542472332656507,
 [src/expression/expr/parse.py:19]:0.7402332101976052,
 [src/expression/expr/parse.py:69]:0.7233642332556182,
 [src/expression/expr/arithmetic.py:20]:0.6914107176547127,
 [src/expression/expr/parse.py:32, src/expression/expr/parse.py:35, src/expression/expr/parse.py:33, src/expression/expr/parse.py:34, src/expression/expr/parse.py:36]:0.6717514421272202,
 [src/expression/expr/parse.py:55]:0.6149186938124421,
 [src/expression/expr/parse.py:47]:0.6030226891555271,
 [src/expression/

In [103]:
faulty_lines = t4p.get_faulty_lines(t4p.expression_1)
faulty_lines

[src/expression/expr/arithmetic.py:33]

In [98]:
assert faulty_lines[0] in suggestions[0].lines